# Deep Learning con redes convolucionales

<a target="_blank" href="https://colab.research.google.com/github/griverat/Meteo-AI/blob/main/notebooks/2.cnn.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Descripción

Este notebook contiene el material a desarrollar durante la sesión de Deep Learning. Se presentarán los conceptos básicos de deep learning y como es que se diferencia principalmente de las redes neuronales tradicionales. Se presentarán ejemplos de aplicaciones y se realizará un ejercicio práctico de clasificación de imágenes.

## Objetivos

- Conocer los conceptos básicos de deep learning.
- Entender cuando es conveniente utilizar deep learning.
- Implementar un modelo de deep learning para clasificación de imágenes.

---

## Sobre el Aprendizaje Profundo

A diferencia de las redes neuronales tradicionales, las redes neuronales profundas (deep learning) son capaces de aprender representaciones jerárquicas de los datos. Esto significa que las redes neuronales profundas pueden aprender a representar los datos en diferentes niveles de abstracción. Por ejemplo, en el caso de imágenes, una red neuronal profunda puede aprender a representar los datos en términos de bordes, formas, texturas, etc.

Debido a la complejidad de las redes neuronales profundas, el perceptrón multicapa (MLP) dejó de ser suficiente para resolver problemas complejos ya que requiere de una gran cantidad de parámetros y por ende una gran cantidad de poder computacional. Como solución a este problema, se desarrollaron las redes neuronales convolucionales (CNN) que son capaces de aprender representaciones jerárquicas de los datos de manera más eficiente.

## Redes Neuronales Convolucionales

<img src="https://upload.wikimedia.org/wikipedia/commons/6/63/Typical_cnn.png"></img>

Arquitectura convolucional típica.
By <a href="//commons.wikimedia.org/w/index.php?title=User:Aphex34&amp;action=edit&amp;redlink=1" class="new" title="User:Aphex34 (page does not exist)">Aphex34</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="https://creativecommons.org/licenses/by-sa/4.0" title="Creative Commons Attribution-Share Alike 4.0">CC BY-SA 4.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=45679374">Link</a>

Una red neuronal convolucional (CNN) es una red neuronal que contiene una o más capas convolucionales. Las capas convolucionales son capaces de aprender representaciones jerárquicas de los datos de manera más eficiente que las redes neuronales tradicionales.

Los bloques básicos de una CNN son:

- **Capa convolucional**: Esta capa aplica una serie de filtros a la entrada y produce un mapa de características. Cada filtro es capaz de aprender diferentes características de los datos.
- **Capa de activación**: Esta capa aplica una función de activación a la salida de la capa convolucional. La función de activación más común es la función ReLU.
- **Capa de agrupación**: Esta capa reduce la dimensionalidad de la entrada aplicando una operación de agrupación. La operación de agrupación más común es la operación de agrupación máxima.
- **Capa completamente conectada**: Esta capa toma la salida de las capas anteriores y produce la salida final de la red.


### Importar librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import xarray as xr

plt.rcParams["font.family"] = "monospace"

### Lectura de datos

Como introducción a las redes neuronales convolucionales, utilizaremos el conjunto de datos MNIST para familiarizarnos con el proceso de entrenamiento de una red neuronal convolucional. El conjunto de datos MNIST contiene imágenes de dígitos escritos a mano y es uno de los conjuntos de datos más utilizados en el campo del aprendizaje automático.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_train.shape

Previsualizamos la data descargada

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(10, 1))
for i in range(10):
    ax[i].imshow(x_train[i], cmap="gray")
    ax[i].axis("off")

Como estamos tratando con imágenes, debemos normalizar los datos para que se encuentren en un rango de 0 a 1. Para ello, dividimos los valores de los píxeles por 255, que es el valor máximo de un píxel.

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1).astype(np.float32) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype(np.float32) / 255.0

Podemos verificar cuales son las dimensiones de nuestros datos

In [ ]:
x_train.shape

Lo que hicimos en este caso fue asegurarnos que la imagen tenga al menos una dimensión de color ademas de la altura y el ancho. En este caso, la imagen tiene una dimensión de color, 28 píxeles de alto y 28 píxeles de ancho. A esta dimension de color se le llama canal.

Ahora que la data de entrada se encuentra lista, nos enfocamos en las etiquetas o datos de salida

In [ ]:
y_train

Podemos verificar que se trata de números enteros que van del 0 al 9, correspondientes a los dígitos escritos a mano. Para poder utilizar estos datos en un modelo de clasificación, necesitamos convertirlos a un formato de one-hot encoding.

**¿Qué es one-hot encoding?**

One-hot encoding es una técnica utilizada en el aprendizaje automático para representar datos categóricos como vectores binarios. En el caso de las etiquetas de los dígitos, cada etiqueta se convierte en un vector binario de 10 elementos, donde el elemento correspondiente a la etiqueta es 1 y los demás elementos son 0.

<img src="https://th.bing.com/th/id/OIP.cnmpSdK-6hAQJdTUBFxcnAHaB3?rs=1&pid=ImgDetMain"></src>

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

Ahora verificamos que las etiquetas se encuentren en el formato correcto

In [ ]:
y_train

### Crear el modelo

Para crear un modelo de red neuronal convolucional, utilizamos la clase Sequential de Keras. La clase Sequential nos permite crear un modelo de red neuronal apilando capas una encima de la otra.

El primer paso para crear un modelo de red neuronal convolucional es definir la arquitectura del modelo. En este caso, utilizaremos una arquitectura simple con dos capas convolucionales seguidas cada una por una capa de agrupación y finalmente una capa completamente conectada.

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

input_shape = (None, 28, 28, 1)
model.build(input_shape)

In [ ]:
model.summary()

### Compilar y entrenar el modelo

Una vez que hemos definido la arquitectura del modelo, debemos compilarlo y entrenarlo. Para compilar el modelo, utilizamos el método compile y especificamos la función de pérdida, el optimizador y las métricas que queremos utilizar para evaluar el modelo.

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
batch_size = 128
epochs = 15

history = model.fit(
    x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1
)

Podemos visualizar la función de perdida

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(history.history["accuracy"], label="train")
ax[0].plot(history.history["val_accuracy"], label="validation")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

ax[1].plot(history.history["loss"], label="train")
ax[1].plot(history.history["val_loss"], label="validation")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()

### Evaluar el modelo

Una vez que hemos entrenado el modelo, podemos evaluar su rendimiento en un conjunto de datos de prueba. Para evaluar el modelo, utilizamos el método evaluate y pasamos el conjunto de datos de prueba y las etiquetas de prueba.

In [ ]:
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

### Predicciones

Finalmente, podemos utilizar el modelo entrenado para hacer predicciones sobre nuevos datos. Para hacer predicciones, utilizamos el método predict y pasamos los datos de entrada sobre los que queremos hacer predicciones.

In [ ]:
idxs = np.random.choice(len(x_test), 25)
images = x_test[idxs]

preds = model.predict(images).argmax(-1)

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i in range(25):
    ax[i // 5, i % 5].imshow(images[i].reshape(28, 28), cmap="gray")
    ax[i // 5, i % 5].axis("off")
    ax[i // 5, i % 5].set_title(
        f"Predicted: {preds[i]}\nTrue: {y_test[idxs[i]].argmax()}"
    )
fig.tight_layout()

Como podemos ver, el modelo es capaz de predecir en un 98% (aproximadamente) de los casos el dígito correcto.
Este es un ejemplo básico de cómo se puede utilizar una red neuronal convolucional para clasificar imágenes.
La base de datos de digitos escritos a mano junto con las etiquetas ya se encontraban en un formato adecuado para ser utilizadas en un modelo de clasificación, requiriendo únicamente de una normalización y un one-hot encoding. En otros casos, usualmente se requiere de un preprocesamiento más complejo de los datos antes de ser utilizados en un modelo de clasificación, como lo vamos a ver en el siguiente ejemplo.

## Pronóstico de la fase de ENOS en el Pacífico Central

En este ejemplo, utilizaremos una red neuronal convolucional para pronosticar la fase de El Niño La Oscilacion Sur (ENOS) en el Pacífico Central. El fenómeno de El Niño es un fenómeno climático que se caracteriza por el calentamiento anómalo de las aguas del Océano Pacífico ecuatorial y que tiene un impacto significativo en el clima global.

Esta demostración tomará como datos de entrada las anomalías de temperatura superficial del mar (SST) en la región del Pacífico durante los últimos 12 meses y pronosticará la fase de El Niño en los próximos 6 meses.

Vamos a descargar los datos de temperatura superficial del mar de Extended Reconstructed Sea Surface Temperature version 5 (ERSSTv5), sobre la cual haremos el cálculo del índice EN3.4 para determinar la fase de ENOS.

In [ ]:
# esta descarga podria tomar algo de tiempo
!mkdir data
!wget https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc -O data/sst.mnmean.nc

Utilizaremos `xarray` para leer los datos de temperatura superficial del mar y calcular el índice EN3.4.

In [ ]:
sst = xr.open_dataset("data/sst.mnmean.nc").sst.sortby("lat")
sst

Para comenzar, los datos deben de estar en terminos de anomalías.

In [ ]:
clim_period = slice("1991", "2020")
sst_clim = sst.sel(time=clim_period).groupby("time.month").mean("time")
sst_clim

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
sst_clim.sel(month=1).plot(ax=ax[0], cmap="inferno")
ax[0].set_title("January")
sst_clim.sel(month=7).plot(ax=ax[1], cmap="inferno")
ax[1].set_title("July")
fig.tight_layout()

El cálculo de las anomalías es trivial usando los métodos de `xarray`.

In [ ]:
sst_anom = sst.groupby("time.month") - sst_clim
sst_anom

Ahora calculamos el índice EN3.4

In [ ]:
en34 = sst_anom.sel(lat=slice(-5, 5), lon=slice(190, 240)).mean(dim=["lat", "lon"])
en34

Podemos visualizar el índice EN3.4 para verificar que se haya calculado correctamente.

In [ ]:
subset = en34.sel(time=slice("1980", None))
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
subset.plot(ax=ax, color="k", linewidth=0.5)
ax.set_title("EN3.4 Index")

ax.fill_between(subset.time, subset, 0.5, where=subset > 0.5, color="red", alpha=0.5)
ax.fill_between(subset.time, subset, -0.5, where=subset < -0.5, color="blue", alpha=0.5)

ax.axhline(0.5, color="red", linestyle="--", lw=0.5)
ax.axhline(-0.5, color="blue", linestyle="--", lw=0.5)
ax.axhline(0, color="black", linestyle="-", lw=0.5)

fig.tight_layout()

`xarray` facilita el uso y manejo de datos climaticos gracias a las funciones de agregación y selección de datos.

Ahora que tenemos las SSTA y el índice EN3.4, podemos proceder a ordenar los datos de acuerdo a la tarea que queremos realizar. primero debemos agrupar los datos en secuencias de 12 meses para las SSTA y 6 meses para el índice EN3.4.

Para lograr esto, vamos a explorar como es que el método `rolling` de `xarray` nos puede ayudar a lograr esto.
`rolling` nos permite crear una ventana movil sobre la cual podemos aplicar funciones de agregación. En nuestro caso, no estamos interesados en agregar los datos, sino en obtener las secuencias consecutivas.

Crearemos un `DataArray` con numeros del 0 al 9 para poder visualizar como funciona el método `rolling`.

In [ ]:
test_data = xr.DataArray(
    np.arange(1, 13),
    dims=["time"],
    coords={"time": pd.date_range("2020-01-01", periods=12)},
)
test_data

In [ ]:
test_data.rolling(time=3)

Para extraer los grupos, debemos construir una nueva dimension en donde sera almacenado

In [ ]:
test_data.rolling(time=3).construct("window_dim")

`rolling` comienza desde la izquierda mientras alinea los datos hacia la derecha, llenando los valores faltantes con `NaN`. Si nos deshacemos de estos valores, obtendremos las secuencias que necesitamos.

Si nos fijamos bien en los tiempos que han sido seleccionados, podemos ver que los dos primeros tiempos tienen valores faltantes y recien a partir del tercer valor se encuentran completos.

In [ ]:
test_data.rolling(time=3).construct("window_dim").isel(time=slice(2, None))

Cabe observar que en este pequeño ejemplo, el mes 03 esta asociado al valor 3 que se encuentra al final de la primera fila, por lo que nuestras etiquetas en el tiempo representan el tiempo final de la secuencia.

In [ ]:
test_data.rolling(time=3).construct("window_dim").isel(time=slice(2, None))[0]

Llevando este ejemplo al set de datos de SSTA con 12 meses, obtenemos lo siguiente

In [ ]:
sst_grouped = sst_anom.rolling(time=12).construct("channel").isel(time=slice(11, None))
sst_grouped

In [ ]:
sst_grouped[0].plot(col="channel", col_wrap=4, cmap="coolwarm", vmin=-3, vmax=3)

Los datos de entrada ya se encuentran agrupados en secuencias de 12 meses, en donde la etiqueta de tiempo corresponde al último mes de la secuencia. Ahora debemos hacer lo mismo con el índice EN3.4. Cabe recordar que el objetivo es pronosticar la fase del ENOS en base al indice EN3.4 en los próximos 6 meses, por lo que primero debemos definir las clases. Por simplicidad, definiremos tres clases: El Niño, La Niña y Neutral, en base a los valores del índice EN3.4.

- Clase 0 - Neutral: -0.5 <= EN3.4 <= 0.5
- Clase 1 - El Niño: EN3.4 > 0.5
- Clase 2 - La Niña: EN3.4 < -0.5

In [ ]:
en34

In [ ]:
enso_class = xr.where(en34 > 0.5, 1, xr.where(en34 < -0.5, 2, 0))
enso_class

In [ ]:
enso_class.tail(100).plot()

Ahora usaremos esta nueva variable de clase para agrupar los datos en secuencias de 6 meses.

In [ ]:
enso_class_grouped = (
    enso_class.rolling(time=6).construct("channel").isel(time=slice(5, None))
)

# ya que rolling asigna el tiempo al ultimo valor de la secuencia, lo corregimos restando 5 meses
# de esta manera el tiempo se asigna al valor inicial de la secuencia
enso_class_grouped["time"] = pd.to_datetime(enso_class_grouped.time) - pd.DateOffset(
    months=5
)

# finalmente, desplazamos la secuencia en un paso hacia adelante
# de esta manera, el valor en el tiempo t, corresponde a los valores t+1, t+2, t+3, t+4, t+5, t+6
enso_class_grouped = enso_class_grouped.shift(time=-1)
enso_class_grouped

Luego de haber realizado todas las operaciones de agrupación, debemos de verificar que los datos esten alineados en el tiempo.

In [ ]:
min_time = max(enso_class_grouped.time.min(), sst_grouped.time.min())
max_time = min(enso_class_grouped.time.max(), sst_grouped.time.max())

sst_grouped_orig = sst_grouped.copy()
sst_grouped = sst_grouped.sel(time=slice(min_time, max_time))
en34_grouped = enso_class_grouped.sel(time=slice(min_time, max_time))

Los datos de entrada no deben de tener valores faltantes, por lo que llenaremos los valores faltantes con 0.

In [ ]:
sst_grouped = sst_grouped.fillna(0)

Exploramos una muestra que sera usada en el modelo

In [ ]:
sst_grouped[0].plot(col="channel", col_wrap=4, cmap="coolwarm", vmin=-3, vmax=3)

In [ ]:
enso_class_grouped[0].plot()

Finalmente, cabe recordar que necesitamos tener estos datos en un formato que el modelo pueda entender, por lo que necesitamos usar one-hot encoding para las etiquetas.

In [ ]:
one_hot_enso = tf.keras.utils.to_categorical(en34_grouped, 3)
one_hot_enso.shape

### Separación de datos

Una vez que hemos agrupado los datos en secuencias de 12 meses para las SSTA y 6 meses para el índice EN3.4, debemos separar los datos en conjuntos de entrenamiento y prueba. Haremos una division de 80/20, donde el 80% de los datos se utilizarán para entrenar el modelo y el 20% restante se utilizará para evaluar el modelo.

In [ ]:
train_size = int(0.8 * len(sst_grouped))

sst_train = sst_grouped.isel(time=slice(None, train_size)).values
enso_train = one_hot_enso[:train_size]

sst_test = sst_grouped.isel(time=slice(train_size, None)).values
enso_test = one_hot_enso[train_size:]

print(f"Train shape: {sst_train.shape}, {enso_train.shape}")
print(f"Test shape: {sst_test.shape}, {enso_test.shape}")

Ahora que tenemos todos los datos preparados, podemos comenzar a construir el modelo de red neuronal convolucional.

### Crear el modelo

Similar al ejemplo anterior con el conjunto de datos MNIST, definimos la arquitectura del modelo con múltiples capas convolucionales seguidas por capas de agrupación y finalmente una capa completamente conectada. En este caso en particular, el modelo tendra 6 salidas, una para cada mes pronosticado.

In [ ]:
def get_model(input_shape):
    model_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(16, (3, 3), activation="relu")(model_input)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)

    x_lead1 = tf.keras.layers.Dense(3, name="lead1")(x)
    x_lead2 = tf.keras.layers.Dense(3, name="lead2")(x)
    x_lead3 = tf.keras.layers.Dense(3, name="lead3")(x)
    x_lead4 = tf.keras.layers.Dense(3, name="lead4")(x)
    x_lead5 = tf.keras.layers.Dense(3, name="lead5")(x)
    x_lead6 = tf.keras.layers.Dense(3, name="lead6")(x)

    model = tf.keras.Model(
        inputs=[model_input],
        outputs=[x_lead1, x_lead2, x_lead3, x_lead4, x_lead5, x_lead6],
    )

    return model

### Compilar y entrenar el modelo

Una vez que hemos definido la arquitectura del modelo, lo compilamos y lo entrenamos con los datos de entrenamiento. En este caso, utilizamos el optimizador Adam y la función de pérdida de entropía cruzada categórica.

In [ ]:
enso_model = get_model(sst_train.shape[1:])
enso_model.summary()

In [ ]:
enso_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"] * 6,
)

batch_size = 64
epochs = 15

history = enso_model.fit(
    sst_train,
    [enso_train[:, i] for i in range(6)],
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    shuffle=True,
)

### Evaluar el modelo

Una vez que hemos entrenado el modelo, evaluamos su rendimiento en el conjunto de datos de prueba. En este caso, utilizamos la precisión como métrica de evaluación.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(history.history["lead1_accuracy"], label="train")
ax[0].plot(history.history["val_lead1_accuracy"], label="validation")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")

ax[1].plot(history.history["loss"], label="train")
ax[1].plot(history.history["val_loss"], label="validation")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")

Hay muchos factores que pueden afectar el rendimiento de la CNN. En este caso puede que los datos que le estemos otorgando a la red tengan demasiado detalle para el tipo de problema que estamos tratando de resolver. En este caso, la red puede estar aprendiendo demasiado sobre los datos de entrenamiento y no generalizando lo suficiente para los datos de prueba.

In [ ]:
score = enso_model.evaluate(sst_test, [enso_test[:, i] for i in range(6)])
print("Test loss:", score[0])

print(f"Lead 1 accuracy: {score[1]*100:.2f}%")
print(f"Lead 2 accuracy: {score[2]*100:.2f}%")
print(f"Lead 3 accuracy: {score[3]*100:.2f}%")
print(f"Lead 4 accuracy: {score[4]*100:.2f}%")
print(f"Lead 5 accuracy: {score[5]*100:.2f}%")
print(f"Lead 6 accuracy: {score[6]*100:.2f}%")

### Prediccion con los valores más recientes

La variable `sst_grouped_orig` contiene los datos mas recientes de SST, por lo que podemos usar estos datos para hacer predicciones sobre la fase de ENOS en los próximos 6 meses.

In [ ]:
latest_data = sst_grouped_orig.isel(time=-1).fillna(0)
latest_data = latest_data.values[None, ...]
latest_data

In [ ]:
# implemente la prediccion de los 6 meses siguientes